## HackViolet 2021

In [1]:

import pandas as pd
import numpy as np
import re
import seaborn as sns
import altair as alt
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [2]:
data = pd.read_csv('OSMI 2019 Mental Health in Tech Survey Results - OSMI Mental Health in Tech Survey 2019.csv')

In [3]:
data.head()

,*Are you self-employed?*,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",...,Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.,"If there is anything else you would like to tell us that has not been covered by the survey questions, please use this space to do so.",Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),What is your age?,What is your gender?,What country do you *live* in?,What US state or territory do you *live* in?,What is your race?,What country do you *work* in?,What US state or territory do you *work* in?
0,False,26-100,True,True,I don't know,No,Yes,Yes,I don't know,Very easy,...,NaN,NaN,False,25,Male,United States of America,Nebraska,White,United States of America,Nebraska
1,False,26-100,True,True,Yes,No,No,Yes,Yes,I don't know,...,NaN,NaN,False,51,male,United States of America,Nebraska,White,United States of America,Nebraska
2,False,26-100,True,True,I don't know,No,No,I don't know,I don't know,Somewhat difficult,...,I think opening up more conversation around th...,Thank you,True,27,Male,United States of America,Illinois,White,United States of America,Illinois
3,False,100-500,True,True,I don't know,No,Yes,Yes,Yes,Very easy,...,NaN,NaN,False,37,male,United States of America,Nebraska,White,United States of America,Nebraska
4,False,26-100,True,True,I don't know,No,I don't know,I don't know,I don't know,I don't know,...,NaN,NaN,False,46,m,United States of America,Nebraska,White,United States of America,Nebraska


In [4]:
data.columns

Index(['*Are you self-employed?*',
       'How many employees does your company or organization have?',
       'Is your employer primarily a tech company/organization?',
       'Is your primary role within your company related to tech/IT?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Do you know the options for mental health care available under your employer-provided health coverage?',
       'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
       'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that lea

In [5]:
data_zeros = data.fillna(-1)

In [6]:
data_zeros["What is your gender?"].value_counts()

Male                                   143
male                                    52
Female                                  49
female                                  27
M                                       15
F                                        9
m                                        8
-1                                       5
Woman                                    4
f                                        3
None                                     2
agender                                  2
Female                                   2
Non-binary                               2
Let's keep it simple and say "male"      1
woman                                    1
Masculine                                1
I am a Wookie                            1
CIS Male                                 1
Trans man                                1
man                                      1
I have a penis                           1
Female-identified                        1
Nonbinary  

In [7]:
data_drop = data_zeros.drop(data_zeros.columns[0:2], axis=1)
data_drop1 = data_drop.drop(data_zeros.columns[13], axis=1)
d_d = data_drop1.drop(data_zeros.columns[29], axis=1)

In [8]:
d_d.shape

(352, 78)

In [9]:
indexNames = d_d[d_d['Is your primary role within your company related to tech/IT?'] == 'False'].index 

In [10]:
indexNames1 = d_d[d_d['Is your primary role within your company related to tech/IT?'] == 0].index 

In [11]:
d_d.drop(indexNames1 , inplace=True)
d_d['Is your primary role within your company related to tech/IT?']

0      True
1      True
2      True
3      True
4      True
       ... 
347    True
348      -1
349      -1
350    True
351    True
Name: Is your primary role within your company related to tech/IT?, Length: 330, dtype: object

In [12]:
d_d["What is your gender?"].value_counts()

Male                                   138
male                                    51
Female                                  44
female                                  22
M                                       14
F                                        8
m                                        7
-1                                       5
Woman                                    4
Non-binary                               2
None                                     2
f                                        2
Female                                   2
agender                                  2
man                                      1
Trans man                                1
cis woman                                1
I am a Wookie                            1
woman                                    1
Nonbinary                                1
Masculine                                1
CIS Male                                 1
I have a penis                           1
Female-iden

In [13]:
data_gendered = d_d

In [14]:
data_gendered["What is your gender?"] = (d_d["What is your gender?"].replace(["Male","male","M","m","CIS Male","man","Cis Male","Identify as male","Make","Trans man","Man","Male ","cis male","Cishet male","Masculine","masculino","I have a penis",'''Let's keep it simple and say "male"'''], "M"))

In [15]:
data_gendered["What is your gender?"] = (data_gendered["What is your gender?"].replace(['Female','female','F','Woman','f','Female','Femile','cis woman','Female-identified','Agender trans woman','Female (cis)','Female ','femmina'], "F"))

In [16]:
data_gendered["What is your gender?"] = (data_gendered["What is your gender?"].replace(['None','agender','Non-binary','Nonbinary','Non binary','Questioning','Trans non-binary/genderfluid',0],"Other"))

In [17]:
data_gendered["What is your gender?"].value_counts()

M                224
F                 88
Other             10
-1                 5
woman              1
rr                 1
I am a Wookie      1
Name: What is your gender?, dtype: int64

In [82]:
more_cleaning = data_gendered[data_gendered.columns.drop(list(data_gendered.filter(regex='Describe')))]
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='Why or')))]
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='describe')))]
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='If you have revealed')))]
more_cleaning = more_cleaning.drop(more_cleaning.columns[22:36], axis=1)
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='If there is anything else')))]
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='Do you believe your productivity')))]
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='Have you ever had a coworker')))]
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='Have you ever discussed')))]
more_cleaning = more_cleaning.drop(more_cleaning.columns[-5:-3], axis=1)
more_cleaning = more_cleaning.drop(more_cleaning.columns[13:18], axis=1)
more_cleaning = more_cleaning.drop(more_cleaning.columns[15], axis=1)
more_cleaning = more_cleaning.drop(more_cleaning.columns[27:29], axis=1)
more_cleaning = more_cleaning[more_cleaning.columns.drop(list(more_cleaning.filter(regex='Have you ever discussed')))]
more_cleaning.columns

Index(['Is your employer primarily a tech company/organization?',
       'Is your primary role within your company related to tech/IT?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Do you know the options for mental health care available under your employer-provided health coverage?',
       'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
       'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?',
       'Would you feel more comfortable talking to your coworkers about your physical health or yo

In [66]:
train_data = more_cleaning[:250]
test_data = more_cleaning[250:]

In [67]:
pca = PCA(n_components = 2)
train_2d = pca.fit_transform(train_data)
kmeans = KMeans(n_clusters = 2, random_state=10) #Set a random state to get the same clusters every time the cell runs
kmeans.fit(train_2d)
kmeans_clusters = kmeans.predict(train_2d)

ValueError: could not convert string to float: "I don't know"

In [145]:
more_clean1 = more_cleaning.replace(to_replace='Yes', value=1)
more_clean2 = more_clean1.replace(to_replace='Yes, I observed', value=1) 
more_clean2 = more_clean2.replace(to_replace='Yes, I experienced',value=1)
more_clean2 = more_clean2.replace(to_replace='No', value=0)
more_clean = more_clean2.replace(to_replace="Don't know", value = 2)
more_clean = more_clean.replace(to_replace="Don't Know", value = 2)
more_clean = more_clean.replace(to_replace='Maybe/Not sure', value=2)
more_clean = more_clean.replace(to_replace='Maybe', value=2)
more_clean = more_clean.replace(to_replace='Possibly', value=2)
more_clean = more_clean.replace(to_replace="I don't know", value = 2)
more_clean = more_clean.replace(to_replace="Not eligible for coverage / NA", value = -1)

In [146]:
more_clean.head()

,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,...,"If they knew you suffered from a mental health disorder, how do you think that your team members/co-workers would react?",Have you observed or experienced an *unsupportive or badly handled response* to a mental health issue in your current or previous workplace?,Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?,"Overall, how well do you think the tech industry supports employees with mental health issues?",Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),What is your age?,What is your gender?,What is your race?,What country do you *work* in?,What US state or territory do you *work* in?
0,True,True,2,0,1,1,2,Very easy,Physical health,1,...,8,0,2,4,False,25,M,White,United States of America,Nebraska
1,True,True,1,0,0,1,1,2,Physical health,2,...,7,1,1,1,False,51,M,White,United States of America,Nebraska
2,True,True,2,0,0,2,2,Somewhat difficult,Same level of comfort for each,0,...,4,2,1,2,True,27,M,White,United States of America,Illinois
3,True,True,2,0,1,1,1,Very easy,Physical health,1,...,8,0,1,3,False,37,M,White,United States of America,Nebraska
4,True,True,2,0,2,2,2,2,Physical health,0,...,5,0,0,3,False,46,M,White,United States of America,Nebraska


In [147]:
more_clean = more_clean.applymap(lambda x: 1 if x == True else x)
more_clean = more_clean.applymap(lambda x: 0 if x == False else x)

In [148]:
more_clean.head()

,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,...,"If they knew you suffered from a mental health disorder, how do you think that your team members/co-workers would react?",Have you observed or experienced an *unsupportive or badly handled response* to a mental health issue in your current or previous workplace?,Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?,"Overall, how well do you think the tech industry supports employees with mental health issues?",Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),What is your age?,What is your gender?,What is your race?,What country do you *work* in?,What US state or territory do you *work* in?
0,1,1,2,0,1,1,2,Very easy,Physical health,1,...,8,0,2,4,0,25,M,White,United States of America,Nebraska
1,1,1,1,0,0,1,1,2,Physical health,2,...,7,1,1,1,0,51,M,White,United States of America,Nebraska
2,1,1,2,0,0,2,2,Somewhat difficult,Same level of comfort for each,0,...,4,2,1,2,1,27,M,White,United States of America,Illinois
3,1,1,2,0,1,1,1,Very easy,Physical health,1,...,8,0,1,3,0,37,M,White,United States of America,Nebraska
4,1,1,2,0,2,2,2,2,Physical health,0,...,5,0,0,3,0,46,M,White,United States of America,Nebraska


In [157]:
more_clean['Have you observed or experienced an *unsupportive or badly handled response* to a mental health issue in your current or previous workplace?']

0      0
1      1
2      2
3      0
4      0
      ..
346    0
347    1
348    1
350    2
351    2
Name: Have you observed or experienced an *unsupportive or badly handled response* to a mental health issue in your current or previous workplace?, Length: 329, dtype: object

In [156]:
more_clean.drop(index=more_clean[more_clean['Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?'] == "I've always been self-employed"].index, inplace=True)


In [105]:
#more_clean.to_csv('mhdf.csv')

In [149]:
medical_leave_map = {"Difficult": 3, "Somewhat difficult": 4, "Neither easy nor difficult": 5,
                     "Somewhat easy": 6, "Very easy": 7, -2:-2, -1:-1}

more_clean['If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?'] = more_clean['If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?'].map(medical_leave_map)

In [153]:
phys_vs_mental_map = {'Physical health': 1, 'Same level of comfort for each': 3, 'Mental health': 2, -1:-1}
more_clean['Would you feel more comfortable talking to your coworkers about your physical health or your mental health?'] = more_clean['Would you feel more comfortable talking to your coworkers about your physical health or your mental health?'].map(phys_vs_mental_map)

In [138]:
more_clean['Would you feel more comfortable talking to your coworkers about your physical health or your mental health?']

0      1
1      1
2      3
3      1
4      1
      ..
347    1
348   -1
349   -1
350    1
351    1
Name: Would you feel more comfortable talking to your coworkers about your physical health or your mental health?, Length: 330, dtype: int64

In [ ]:
gen_ord_map = {'Gen 1': 1, 'Gen 2': 2, 'Gen 3': 3, 
               'Gen 4': 4, 'Gen 5': 5, 'Gen 6': 6}
poke_df['GenerationLabel'] = poke_df['Generation'].map(gen_ord_map)
poke_df[['Name', 'Generation', 'GenerationLabel']].iloc[4:10]

In [154]:
more_clean.value_counts()

Is your employer primarily a tech company/organization?  Is your primary role within your company related to tech/IT?  Does your employer provide mental health benefits as part of healthcare coverage?  Do you know the options for mental health care available under your employer-provided health coverage?  Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?  Does your employer offer resources to learn more about mental health disorders and options for seeking help?  Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?  If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?  Would you feel more comfortable talking to your coworkers about your physical health or your mental health?  Would you feel comfortable discussing a mental health issue w

In [158]:
more_clean["If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*"].value_counts()

Not applicable to me    124
Sometimes               105
Rarely                   67
Often                    25
Never                     8
Name: If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*, dtype: int64

In [159]:
more_clean['If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*'].value_counts()

Often                   136
Not applicable to me    107
Sometimes                71
Rarely                   12
Never                     3
Name: If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*, dtype: int64

In [160]:
interference_map = {'Not applicable to me': 0, 'Never': 1, 'Rarely': 2, 'Sometimes': 3, 'Often': 4}
more_clean["If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*"] = more_clean["If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*"].map(interference_map)

In [161]:
more_clean["If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*"]

0      0
1      3
2      0
3      0
4      0
      ..
346    0
347    3
348    0
350    4
351    3
Name: If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*, Length: 329, dtype: int64

In [162]:
more_clean['If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*'] = more_clean['If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*'].map(interference_map)

In [163]:
more_clean['If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*']

0      0
1      4
2      0
3      0
4      0
      ..
346    4
347    3
348    0
350    4
351    4
Name: If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*, Length: 329, dtype: int64

In [164]:
more_clean.head()

,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,...,"If they knew you suffered from a mental health disorder, how do you think that your team members/co-workers would react?",Have you observed or experienced an *unsupportive or badly handled response* to a mental health issue in your current or previous workplace?,Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?,"Overall, how well do you think the tech industry supports employees with mental health issues?",Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),What is your age?,What is your gender?,What is your race?,What country do you *work* in?,What US state or territory do you *work* in?
0,1,1,2,0,1,1,2,7.0,1,1,...,8,0,2,4,0,25,M,White,United States of America,Nebraska
1,1,1,1,0,0,1,1,NaN,1,2,...,7,1,1,1,0,51,M,White,United States of America,Nebraska
2,1,1,2,0,0,2,2,4.0,3,0,...,4,2,1,2,1,27,M,White,United States of America,Illinois
3,1,1,2,0,1,1,1,7.0,1,1,...,8,0,1,3,0,37,M,White,United States of America,Nebraska
4,1,1,2,0,2,2,2,NaN,1,0,...,5,0,0,3,0,46,M,White,United States of America,Nebraska
